In [1]:
import os
import pandas as pd
from tqdm import tqdm
from classes.vqa_model_predictor import DefaultVqaModelPredictor
from evaluate.VqaMedEvaluatorBase import VqaMedEvaluatorBase
from common.utils import VerboseTimer
import vqa_logger 
import logging
from pathlib import Path
import datetime
logger = logging.getLogger(__name__)

Using TensorFlow backend.


In [2]:
%%capture
mp = DefaultVqaModelPredictor.get_contender()

[2019-04-09 11:00:20][common.utils][DEBUG] Starting 'Loading  VQA contender'
[2019-04-09 11:00:20][classes.vqa_model_predictor][DEBUG] Getting model #78 (post_concat_dense_units: 8;
Optimizer: RMSprop
loss: cosine_proximity
activation: sigmoid
prediction vector: words
lstm_units: 0
batch_size: 75
Based on #72, with class weights

model_location = 'C:\Users\Public\Documents\Data\2019\models\20190409_1040_43\vqa_model.h5')
[2019-04-09 11:00:20][common.utils][DEBUG] Starting 'Loading Model'
[2019-04-09 11:00:24][common.utils][DEBUG] Loading Model: 0:00:04.127377
[2019-04-09 11:00:24][classes.vqa_model_predictor][WARNING] Expected main model to be with no question category, but got: "Abnormality"
[2019-04-09 11:00:24][classes.vqa_model_predictor][DEBUG] Getting model #78 (post_concat_dense_units: 8;
Optimizer: RMSprop
loss: cosine_proximity
activation: sigmoid
prediction vector: words
lstm_units: 0
batch_size: 75
Based on #72, with class weights

model_location = 'C:\Users\Public\Documents

In [3]:
mp

VqaModelPredictor(model=78, specialized_classifiers={'Abnormality': 78, 'Modality': 69, 'Organ': 70, 'Plane': 71})

In [4]:
%%capture
datasets = {'test':mp.df_test, 'validation':mp.df_validation}
df_name_to_predict = 'test'


predictions = {}

for name, df in datasets.items():
    with VerboseTimer(f"Predictions for VQA contender {name}"):
        df_predictions = mp.predict(df)
        predictions[name] = df_predictions



predictions['validation'][:5]

[2019-04-09 11:00:33][common.utils][DEBUG] Starting 'Predictions for VQA contender test'
[2019-04-09 11:00:33][classes.vqa_model_predictor][INFO] For Category "Abnormality" using specialized classifier from:
ModelFolder(folder="C:\\Users\\Public\\Documents\\Data\\2019\\models\\20190409_1040_43")
[2019-04-09 11:00:33][classes.vqa_model_predictor][DEBUG] Classifying: "Abnormality"
[2019-04-09 11:00:33][common.functions][DEBUG] Getting image features
[2019-04-09 11:00:33][common.utils][DEBUG] Starting 'Raw model prediction'
[2019-04-09 11:00:40][common.utils][DEBUG] Raw model prediction: 0:00:06.529068
[2019-04-09 11:00:41][classes.vqa_model_predictor][INFO] For Category "Modality" using specialized classifier from:
ModelFolder(folder="C:\\Users\\Public\\Documents\\Data\\2019\\models\\20190329_0133_06")
[2019-04-09 11:00:41][classes.vqa_model_predictor][DEBUG] Classifying: "Modality"
[2019-04-09 11:00:41][common.functions][DEBUG] Getting image features
[2019-04-09 11:00:41][common.utils][

In [5]:
outputs = {}
for name, df_predictions in predictions.items():
    curr_predictions = df_predictions.prediction.values
    df_predicted = datasets[name]
    df_output = df_predicted.copy()
    df_output['image_id'] = df_output.path.apply(lambda p: p.rsplit(os.sep)[-1].rsplit('.', 1)[0])
    df_output['prediction'] = curr_predictions

    columns_to_remove = ['path',  'answer_embedding', 'question_embedding', 'group', 'diagnosis', 'processed_answer']
    for col in columns_to_remove:
        del df_output[col]

    sort_columns = sorted(df_output.columns, key=lambda c: c not in ['question', 'prediction', 'answer'])
    df_output = df_output[sort_columns]    
    outputs[name] = df_output

In [6]:
df_output_test = outputs['test']
df_output_validation = outputs['validation']

In [7]:
display = df_output_validation[df_output_validation.question_category == 'Abnormality']
display = df_output_validation
display.sample(10)

,question,answer,prediction,image_name,processed_question,question_category,image_id
13550,which plane is the image shown in?,longitudinal,transverse,synpic27433.jpg,which plane is the image shown in,Plane,synpic27433
13247,what imaging modality was used to take this im...,mr - t1w w/gadolinium,mr t2 weighted,synpic54675.jpg,what imaging modality was used to take this image,Modality,synpic54675
13097,is this a contrast or noncontrast ct?,noncontrast,noncontrast,synpic28295.jpg,is this contrast or noncontrast ct,Modality,synpic28295
14597,what abnormality is seen in the image?,mucocele,abscess cerebral glioblastoma multiforme menin...,synpic28295.jpg,what abnormality is seen in the image,Abnormality,synpic28295
13552,in what plane was this image taken?,axial,axial,synpic42268.jpg,in what plane was this image taken,Plane,synpic42268
14538,what is most alarming about this mri?,tricholemmoma,cyst astrocytoma acute cerebral cell,synpic45466.jpg,what is most alarming about this mri,Abnormality,synpic45466
14083,what is the organ principally shown in this ul...,vascular and lymphatic,skull and contents,synpic25374.jpg,what is the organ principally shown in this ul...,Organ,synpic25374
13992,what organ system is evaluated primarily?,skull and contents,skull and contents,synpic54169.jpg,what organ system is evaluated primarily,Organ,synpic54169
13327,in what plane is this image taken?,axial,axial,synpic19169.jpg,in what plane is this image taken,Plane,synpic19169
12929,what type of image modality is this?,an - angiogram,mr other pulse seq,synpic33236.jpg,what type of image modality is this,Modality,synpic33236


In [8]:
mp

VqaModelPredictor(model=78, specialized_classifiers={'Abnormality': 78, 'Modality': 69, 'Organ': 70, 'Plane': 71})

In [9]:
len(df_output_test), len(df_output_test.image_id.drop_duplicates())

(500, 500)

In [10]:
def get_str(df):
    strs = []
    debug_output_rows = df.apply(lambda row: row.image_id + '|'+ row.question + '|'+ row.prediction, axis=1 )
    output_rows = df.apply(lambda row: row.image_id + '|'+ row.prediction + '|'+row.answer, axis=1 )
    output_rows = output_rows.str.strip('|')
    rows = output_rows.values
    res = '\n'.join(rows)
    return res

res = get_str(df_output_test)
#res_val = get_str(df_output_validation)


In [11]:
print(res[:200])

synpic54082|cta ct angiography
synpic48556|no
synpic50696|iv
synpic37194|us doppler ultrasound
synpic31308|an angiogram
synpic24739|t1
synpic32365|yes
synpic47936|iv
synpic29792|yes
synpic43561|no
syn


### Get evaluations per category:

In [12]:
evaluations = {}
pbar = tqdm(df_output_validation.groupby('question_category'))
for question_category, df in pbar:        
    pbar.set_description(f'evaluating {len(df)} for {question_category} items')
    curr_predictions = df.prediction.values
    curr_ground_truth = df.answer.values
    curr_evaluations = VqaMedEvaluatorBase.get_all_evaluation(predictions=curr_predictions, ground_truth=curr_ground_truth)
    evaluations[question_category] = curr_evaluations    

   


evaluating 500 for Plane items: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:09<00:00,  3.12s/it]


### Get Total Evaluation:

In [13]:
total_evaluations = VqaMedEvaluatorBase.get_all_evaluation(predictions=df_output_validation.prediction.values, ground_truth=df_output_validation.answer.values)    
evaluations['Total'] = total_evaluations

In [14]:
evaluations
df_evaluations = pd.DataFrame(evaluations).T#.sort_values(by=('bleu'))
df_evaluations['sort'] = df_evaluations.index == 'Total'
df_evaluations = df_evaluations.sort_values(by = ['sort', 'wbss'])
del df_evaluations['sort']
df_evaluations

,bleu,wbss
Abnormality,0.165479,0.155145
Organ,0.708409,0.699596
Modality,0.763459,0.713040
Plane,0.739682,0.743336
Total,0.594257,0.577779


In [15]:
model_repr = repr(mp)
model_repr
sub_models = {category: folder for category, (model, folder) in mp.model_by_question_category.items()}
sub_models_str = '\n'.join([str(f'{category}: {folder} ({folder.prediction_data_name})') for category, folder in sub_models.items() if folder is not None])

model_description_copy = df_evaluations.copy()

def get_prediction_vector(category):
    sub_model = sub_models.get(category)
    if sub_model is not None:
        return sub_model.prediction_data_name
    else:
        return '--'
    
model_description_copy['prediction_vector'] = model_description_copy.index.map(get_prediction_vector)


model_description =f'''
==Model==
{model_repr}

==Submodels==
{sub_models_str}

==validation evaluation==
{model_description_copy.to_string()}
'''


print(model_description)




==Model==
VqaModelPredictor(model=78, specialized_classifiers={'Abnormality': 78, 'Modality': 69, 'Organ': 70, 'Plane': 71})

==Submodels==
Abnormality: ModelFolder(folder="C:\\Users\\Public\\Documents\\Data\\2019\\models\\20190409_1040_43") (words)
Modality: ModelFolder(folder="C:\\Users\\Public\\Documents\\Data\\2019\\models\\20190329_0133_06") (answers)
Organ: ModelFolder(folder="C:\\Users\\Public\\Documents\\Data\\2019\\models\\20190329_0440_18") (answers)
Plane: ModelFolder(folder="C:\\Users\\Public\\Documents\\Data\\2019\\models\\20190329_0647_26") (answers)

==validation evaluation==
                 bleu      wbss prediction_vector
Abnormality  0.165479  0.155145             words
Organ        0.708409  0.699596           answers
Modality     0.763459  0.713040           answers
Plane        0.739682  0.743336           answers
Total        0.594257  0.577779                --



In [16]:
import time
now = time.time()
ts = datetime.datetime.fromtimestamp(now).strftime('%Y%m%d_%H%M_%S')
submission_base_folder = Path('C:\\Users\\Public\\Documents\\Data\\2019\\submissions')
submission_folder = submission_base_folder/ts
submission_folder.mkdir()


txt_path = submission_folder/f'submission_{ts}.txt'
txt_path.write_text(res)


model_description_path = submission_folder/f'model_description.txt'
model_description_path.write_text(model_description)

with pd.HDFStore(str(submission_folder/ 'predictions.hdf')) as store:
    for name, df_predictions in predictions.items():
        store[name] = df_predictions

In [17]:
print(name)
df_predictions.sample(5)

validation


,image_name,question,answer,prediction,probabilities,path
index,,,,,,
13400,synpic27730.jpg,what plane is this film,transverse,coronal,[(0.163)],C:\Users\Public\Documents\Data\2019\validation...
12978,synpic56608.jpg,what imaging modality was used to take this im...,ct noncontrast,ct noncontrast,[(0.000)],C:\Users\Public\Documents\Data\2019\validation...
14396,synpic17620.jpg,what is most alarming about this ct scan?,cholangiocarcinoma,carcinoma breast adenocarcinoma lung gastroint...,"[(0.000), (0.000), (0.000), (0.000), (0.000)]",C:\Users\Public\Documents\Data\2019\validation...
14763,synpic58887.jpg,what abnormality is seen in the image?,bone metastasis,cyst cerebral aneurysm spinal fracture,"[(0.000), (0.000), (0.000), (0.000), (0.000)]",C:\Users\Public\Documents\Data\2019\validation...
13719,synpic50775.jpg,what plane was used in this mri?,sagittal,axial,[(0.133)],C:\Users\Public\Documents\Data\2019\validation...


In [18]:
idx = 14525
series = df_predictions.loc[idx]
series.answer, series.prediction ,series.probabilities 

('pericardial cyst',
 'multiforme glioblastoma abscess meningioma cerebral',
 array(['(0.003), (0.003), (0.002), (0.002), (0.002)'], dtype=object))

In [19]:
mp

VqaModelPredictor(model=78, specialized_classifiers={'Abnormality': 78, 'Modality': 69, 'Organ': 70, 'Plane': 71})